In [0]:
df = spark.table("dados_covid_2022").alias("dc")
total = df.count()
print(f"total de linhas: {total}")


total de linhas: 999054


In [0]:
print(f"schemas = {df.columns}")
df.printSchema()

schemas = ['ID', 'ORIGEM', 'DT_NOTIFIC', 'DT_NASC', 'IDADE', 'SEXO', 'CS_RACA', 'DT_SIN_PRI', 'SEM_PRI', 'FEBRE', 'TOSSE', 'GARGANTA', 'DISPNEIA', 'DESC_RESP', 'SATURACAO', 'DIARREIA', 'VOMITO', 'DOR_ABD', 'CORIZA', 'CABECA', 'PERD_OLFT', 'PERD_PALA', 'OUTRO_SIN', 'GESTANTE', 'PUERPERA', 'CARDIOPATI', 'HEMATOLOGI', 'SIND_DOWN', 'HEPATICA', 'ASMA', 'DIABETES', 'NEUROLOGIC', 'PNEUMOPATI', 'IMUNODEPRE', 'RENAL', 'OBESIDADE', 'CROMOSSOMO', 'OUT_MORBI', 'HOSPITAL', 'DT_INTERNA', 'CO_UN_INTE', 'NM_UN_INTE', 'SRAG_CLASSI_FIN', 'CLASSI_FIN_COVID', 'CLASSI_FIN_COVID_MOD', 'CRITERIO', 'EVOL_COMUM', 'DT_EVOLUCA', 'SEM_EVOLUCA', 'DT_DIGITA']
root
 |-- ID: string (nullable = true)
 |-- ORIGEM: string (nullable = true)
 |-- DT_NOTIFIC: string (nullable = true)
 |-- DT_NASC: string (nullable = true)
 |-- IDADE: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- CS_RACA: string (nullable = true)
 |-- DT_SIN_PRI: string (nullable = true)
 |-- SEM_PRI: string (nullable = true)
 |-- FEBRE:

In [0]:
df_update = df.withColumnsRenamed({"DT_SIN_PRI":  "DATA_INICIO_SINT", "DT_INTERNA": "DATA_INTERNACAO", "DT_EVOLUCA": "DATA_EVOLUCAO", "FEBRE":  "teve_febre", "SEXO": "sexo", "TOSSE":  "teve_tosse", "HOSPITAL":  "paciente_internado", "CS_RACA": "RACA"})
df_update.printSchema()

root
 |-- ID: string (nullable = true)
 |-- ORIGEM: string (nullable = true)
 |-- DT_NOTIFIC: string (nullable = true)
 |-- DT_NASC: string (nullable = true)
 |-- IDADE: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- RACA: string (nullable = true)
 |-- DATA_INICIO_SINT: string (nullable = true)
 |-- SEM_PRI: string (nullable = true)
 |-- teve_febre: string (nullable = true)
 |-- teve_tosse: string (nullable = true)
 |-- GARGANTA: string (nullable = true)
 |-- DISPNEIA: string (nullable = true)
 |-- DESC_RESP: string (nullable = true)
 |-- SATURACAO: string (nullable = true)
 |-- DIARREIA: string (nullable = true)
 |-- VOMITO: string (nullable = true)
 |-- DOR_ABD: string (nullable = true)
 |-- CORIZA: string (nullable = true)
 |-- CABECA: string (nullable = true)
 |-- PERD_OLFT: string (nullable = true)
 |-- PERD_PALA: string (nullable = true)
 |-- OUTRO_SIN: string (nullable = true)
 |-- GESTANTE: string (nullable = true)
 |-- PUERPERA: string (nullable = true)
 |--

In [0]:
from pyspark.sql.functions import when

df_update_febres = df_update.withColumn("teve_febre", when(df_update["teve_febre"] == 1, "sim").when(df_update["teve_febre"] == 2, "nao").otherwise("ignorado"))

df_update_febres.groupBy(["sexo","teve_febre"]).count().sort("sexo").show()

+----+----------+------+
|sexo|teve_febre| count|
+----+----------+------+
|   F|       nao|  2450|
|   F|  ignorado|438097|
|   F|       sim|179034|
|   M|  ignorado|248124|
|   M|       nao|  2164|
|   M|       sim|129185|
+----+----------+------+



In [0]:
from pyspark.sql.functions import when
df_update_tosse = df_update_febres.withColumn("teve_tosse", when(df_update_febres["teve_tosse"] == 1, "sim").otherwise("nao"))

df_update_tosse.groupBy(["sexo","teve_tosse"]).count().sort("sexo").show()

+----+----------+------+
|sexo|teve_tosse| count|
+----+----------+------+
|   F|       nao|224614|
|   F|       sim|394967|
|   M|       nao|140384|
|   M|       sim|239089|
+----+----------+------+



In [0]:
from pyspark.sql.functions import when

df_update_hospital = df_update_tosse.withColumn("paciente_internado", when(df_update_tosse["paciente_internado"] == "1", "sim").otherwise("nao"))

df_update_hospital.groupBy(["sexo","paciente_internado"]).count().sort("sexo").show()

+----+------------------+------+
|sexo|paciente_internado| count|
+----+------------------+------+
|   F|               nao|611428|
|   F|               sim|  8153|
|   M|               nao|371475|
|   M|               sim|  7998|
+----+------------------+------+



In [0]:
total_paciente = df_update_hospital.filter("paciente_internado = 'sim'").count()

print(f"total de linhas: {total_paciente}")

total de linhas: 16151


In [0]:

from pyspark.sql.functions import datediff

df_diff_internacao_evolucao = df_update.select("sexo","DATA_INTERNACAO",
"DATA_EVOLUCAO", datediff(df_update.DATA_EVOLUCAO, df_update.DATA_INTERNACAO).alias("dias_evolucao"))

df_diff_internacao_evolucao.filter(df_update["DATA_INTERNACAO"].isNotNull() & df_update["DATA_EVOLUCAO"].isNotNull()).show()


+----+---------------+-------------+-------------+
|sexo|DATA_INTERNACAO|DATA_EVOLUCAO|dias_evolucao|
+----+---------------+-------------+-------------+
|   M|     2022-06-30|   2022-07-03|            3|
|   M|     2022-06-29|   2022-06-30|            1|
|   M|     2022-06-15|   2022-06-23|            8|
|   M|     2022-06-14|   2022-06-22|            8|
|   F|     2022-06-27|   2022-07-15|           18|
|   F|     2022-06-29|   2022-07-01|            2|
|   F|     2022-06-28|   2022-07-16|           18|
|   F|     2022-06-17|   2022-06-26|            9|
|   M|     2022-05-23|   2022-06-03|           11|
|   F|     2022-06-19|   2022-07-08|           19|
|   F|     2022-05-09|   2022-06-20|           42|
|   F|     2022-06-18|   2022-06-22|            4|
|   M|     2022-06-14|   2022-06-19|            5|
|   F|     2022-06-28|   2022-07-03|            5|
|   M|     2022-06-19|   2022-06-25|            6|
|   F|     2022-06-23|   2022-07-01|            8|
|   F|     2022-06-28|   2022-0

In [0]:

from pyspark.sql.functions import max

df_diff_internacao_evolucao.groupBy(["sexo"]).agg(max("dias_evolucao").alias("dias_evolucao_maior")).show()

+----+-------------------+
|sexo|dias_evolucao_maior|
+----+-------------------+
|   F|                696|
|   M|                852|
+----+-------------------+



In [0]:
from pyspark.sql.functions import min

df_evolucao_filter = df_diff_internacao_evolucao.filter(df_diff_internacao_evolucao["dias_evolucao"] > 0)

df_evolucao_min = df_evolucao_filter.groupBy(["sexo"]).agg(min("dias_evolucao").alias("dias_evolucao_menor")).show()



+----+-------------------+
|sexo|dias_evolucao_menor|
+----+-------------------+
|   F|                  1|
|   M|                  1|
+----+-------------------+



In [0]:
from pyspark.sql.functions import when

df_update_raca = df_update_hospital.withColumn("RACA", when(df_update_hospital["RACA"] == "1", "branca").when(df_update_hospital["RACA"] == "2", "preta").when(df_update_hospital["RACA"] == "3", "amarela").when(df_update_hospital["RACA"] == "4", "parda").when(df_update_hospital["RACA"] == "5", "indigena").when(df_update_hospital["RACA"] == "9", "ignorado").otherwise("NULL"))

df_raca_filter = df_update_raca.filter(df_update_raca["RACA"] != "NULL")

df_raca_filter.groupBy(["RACA","sexo"]).count().show()

+--------+----+------+
|    RACA|sexo| count|
+--------+----+------+
|indigena|   M|   161|
|   preta|   M| 18294|
|ignorado|   F|179121|
| amarela|   F| 14055|
|  branca|   M|117948|
|indigena|   F|   216|
|ignorado|   M|111490|
|   preta|   F| 28095|
| amarela|   M|  8799|
|  branca|   F|202608|
|   parda|   M|122777|
|   parda|   F|195482|
+--------+----+------+



In [0]:
df_raca_filter.groupBy(["RACA"]).count().show()

+--------+------+
|    RACA| count|
+--------+------+
|   parda|318259|
|ignorado|290611|
| amarela| 22854|
|  branca|320556|
|indigena|   377|
|   preta| 46389|
+--------+------+



In [0]:
from pyspark.sql.functions import lit

df_new_colunm = df_raca_filter.withColumn("ano_refe", lit("2022"))
df_new_colunm.select("ano_refe").show()

+--------+
|ano_refe|
+--------+
|    2022|
|    2022|
|    2022|
|    2022|
|    2022|
|    2022|
|    2022|
|    2022|
|    2022|
|    2022|
|    2022|
|    2022|
|    2022|
|    2022|
|    2022|
|    2022|
|    2022|
|    2022|
|    2022|
|    2022|
+--------+
only showing top 20 rows



In [0]:
df_new_colunm.write.parquet("dbfs:/FileStore/covid/ano_ref=2022/")

In [0]:
dbutils.fs.ls("/FileStore/covid/ano_ref=2022/")

[FileInfo(path='dbfs:/FileStore/coiv/ano_ref=2022/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1737505751000),
 FileInfo(path='dbfs:/FileStore/coiv/ano_ref=2022/_committed_3958400291199506185', name='_committed_3958400291199506185', size=824, modificationTime=1737505751000),
 FileInfo(path='dbfs:/FileStore/coiv/ano_ref=2022/_started_3958400291199506185', name='_started_3958400291199506185', size=0, modificationTime=1737505720000),
 FileInfo(path='dbfs:/FileStore/coiv/ano_ref=2022/part-00000-tid-3958400291199506185-e963d354-954c-482b-9c96-dfb98e77067c-145-1-c000.snappy.parquet', name='part-00000-tid-3958400291199506185-e963d354-954c-482b-9c96-dfb98e77067c-145-1-c000.snappy.parquet', size=2121526, modificationTime=1737505750000),
 FileInfo(path='dbfs:/FileStore/coiv/ano_ref=2022/part-00001-tid-3958400291199506185-e963d354-954c-482b-9c96-dfb98e77067c-146-1-c000.snappy.parquet', name='part-00001-tid-3958400291199506185-e963d354-954c-482b-9c96-dfb98e77067c-146-1-c000.snappy.parquet'